In [34]:
## classify cats

import numpy as np

best_feature_vectors = np.load("features_train.npy")
test_feature_vectors = np.load("features_test.npy")
lengths = np.load("category_lengths.npy")
category_info = np.load("words_in_categories.npy")

toSelect = 5
tEx = 10
training_amt = 8
testing_amt = 2

trainx = np.zeros((0,toSelect * tEx))
trainy = np.zeros((training_amt * tot_words))
testx = np.zeros((0, toSelect * tEx))
testy = np.zeros((testing_amt * tot_words))

ytraincnt = 0
ytestcnt = 0

for pres in range(training_amt):
    for word in range(63):
        trainx = np.concatenate((trainx, np.reshape(best_feature_vectors[word][pres],(1,toSelect*tEx))), axis=0)
        trainy[ytraincnt] = [m for m in range(12) if word in category_info[m]][0]
        ytraincnt+=1

        if(pres<testing_amt):
            testx = np.concatenate((testx, np.reshape(test_feature_vectors[word][pres],(1,toSelect*tEx))), axis=0)
            testy[ytestcnt] = [m for m in range(12) if word in category_info[m]][0]
            ytestcnt+=1           
            
#if you want to convert to one-hot encoding, uncomment the 2 lines below:
#trainy = keras.utils.to_categorical(trainy, num_classes=12)
#testy = keras.utils.to_categorical(testy, num_classes=12)

In [35]:
trainy.shape

(504,)

In [38]:
import numpy as np
from scipy.stats import pearsonr
from heapq import heappush, heappop, heappushpop
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import itertools
import pickle

#clist = np.logspace(-4,2,100)

training_amt = 8 #num of presentations used for training
testing_amt = 2 #num of presentations used for testing
toSelect = 5 #num of btc's selected
tEx = 10 #number of features per BTC vector

avgacc = 0

bst_acc = 0
bst_c = 1


for c in clist:
    avg_acc = 0
    
    for fold in range(4):
        fold_sz = int(trainx.shape[0]/4)
        valid_x = trainx[(fold_sz*fold):((fold_sz)*(fold+1))]
        valid_y = trainy[(fold_sz*fold):((fold_sz)*(fold+1))]
        tr_x = np.concatenate((trainx[:(fold_sz*fold)],trainx[((fold+1)*fold_sz):]), axis = 0)
        tr_y = np.concatenate((trainy[:(fold_sz*fold)],trainy[((fold+1)*fold_sz):]), axis = 0)

        scaler = StandardScaler()
        tr_x = scaler.fit_transform(tr_x)
        valid_x = scaler.transform(valid_x)
        tr_y = np.ravel(tr_y)
        valid_y = np.ravel(valid_y)

        classifier = LinearSVC(C = c)
        classifier.fit(tr_x,tr_y)
        avg_acc += (classifier.score(valid_x, valid_y))/4.0
    if(avg_acc > bst_acc):
        bst_acc = avg_acc
        bst_c = c

    
clf = LinearSVC(C=bst_c, multi_class='ovr')
scaler = StandardScaler()
trainx = scaler.fit_transform(trainx)
testx = scaler.transform(testx)
trainy = np.ravel(trainy)
testy = np.ravel(testy)
clf.fit(trainx, trainy)
myscore = clf.score(testx, testy)
avgacc+=myscore

#print("For " + str(pair[0]) + " and " + str(pair[1]) + " we picked C = " + str(bst_c))
#print("Has accuracy " + str(myscore))
#print("=========")

print(myscore)

0.214285714286


In [17]:
import numpy as np
from scipy.stats import pearsonr
from heapq import heappush, heappop, heappushpop
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import itertools
import pickle

file_name = 'CategoryXCategory.p' #change this appropriately

loaded_data = pickle.load(open(file_name,"rb"))

dict_trainx = loaded_data[0]
dict_trainy = loaded_data[1]
dict_testx = loaded_data[2]
dict_testy = loaded_data[3]

clist = np.logspace(-4,2,100)

training_amt = 8 #num of presentations used for training
testing_amt = 2 #num of presentations used for testing
toSelect = 5 #num of btc's selected
tEx = 10 #number of features per BTC vector

avgacc = 0
for pair in dict_trainx:
    trainx = dict_trainx[pair]
    trainy = dict_trainy[pair]
    testx = dict_testx[pair]
    testy = dict_testy[pair]

    bst_acc = 0
    bst_c = 0

    for c in clist:
        avg_acc = 0
        for fold in range(4):
            fold_sz = int(trainx.shape[0]/4)
            valid_x = trainx[(fold_sz*fold):((fold_sz)*(fold+1))]
            valid_y = trainy[(fold_sz*fold):((fold_sz)*(fold+1))]
            tr_x = np.concatenate((trainx[:(fold_sz*fold)],trainx[((fold+1)*fold_sz):]), axis = 0)
            tr_y = np.concatenate((trainy[:(fold_sz*fold)],trainy[((fold+1)*fold_sz):]), axis = 0)

            scaler = StandardScaler()
            tr_x = scaler.fit_transform(tr_x)
            valid_x = scaler.transform(valid_x)
            tr_y = np.ravel(tr_y)
            valid_y = np.ravel(valid_y)

            classifier = LinearSVC(C = c)
            classifier.fit(tr_x,tr_y)
            avg_acc += (classifier.score(valid_x, valid_y))/4.0
        if(avg_acc > bst_acc):
            bst_acc = avg_acc
            bst_c = c
    clf = LinearSVC(C=bst_c)
    scaler = StandardScaler()
    trainx = scaler.fit_transform(trainx)
    testx = scaler.transform(testx)
    trainy = np.ravel(trainy)
    testy = np.ravel(testy)
    clf.fit(trainx, trainy)
    myscore = clf.score(testx, testy)
    avgacc+=myscore

    print("For " + str(pair[0]) + " and " + str(pair[1]) + " we picked C = " + str(bst_c))
    print("Has accuracy " + str(myscore))
    print("=========")

For 5 and 9 we picked C = 1.0
Has accuracy 0.6
For 4 and 7 we picked C = 75.6463327555
Has accuracy 0.8
For 1 and 3 we picked C = 12.3284673944
Has accuracy 0.7
For 10 and 11 we picked C = 49.7702356433
Has accuracy 0.85
For 4 and 8 we picked C = 16.2975083462
Has accuracy 0.65
For 5 and 6 we picked C = 0.657933224658
Has accuracy 0.75
For 2 and 8 we picked C = 6.13590727341
Has accuracy 0.666666666667
For 3 and 11 we picked C = 5.33669923121
Has accuracy 0.7
For 0 and 7 we picked C = 1.51991108295
Has accuracy 0.772727272727
For 4 and 6 we picked C = 18.7381742286
Has accuracy 0.6
For 8 and 9 we picked C = 14.1747416293
Has accuracy 0.65
For 1 and 6 we picked C = 86.9749002618
Has accuracy 0.6
For 0 and 10 we picked C = 43.2876128108
Has accuracy 0.727272727273
For 3 and 7 we picked C = 4.0370172586
Has accuracy 0.65
For 7 and 11 we picked C = 24.7707635599
Has accuracy 0.7
For 0 and 3 we picked C = 0.572236765935
Has accuracy 0.863636363636
For 1 and 11 we picked C = 0.0613590727341


In [18]:
avgacc

44.365384615384606

In [19]:
avgacc /= (12*11/2)

In [20]:
avgacc

0.67220279720279708